In [1]:
# PyTorch and tensor operations
import torch

# Mathematical operations
from math import exp, factorial

# Plotting
import matplotlib.pyplot as plt

# Progress tracking
from tqdm import tqdm

# File handling and JSON processing
import json
import os
import time

# Data manipulation
import pandas as pd

# Table formatting for better visualization
from tabulate import tabulate
from torch import tensor

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [3]:
# tensor = tensor.to(device)

In [4]:
# print(f"Device being used: {device}")

In [5]:
# your_tensor_variable = your_tensor_variable.to(device)

## Experimental Parameters

In [6]:
# e_1
# Fiber lengths
Ls = torch.linspace(0.1, 200, 1000)  # Fiber lengths in km
L_BC = Ls
e_1 = L_BC / 100

#e_2
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
P_dc_value = torch.tensor(6e-7, device=device)  # Dark count probability
Y_0 = P_dc_value
# 2.7*10** -7
# P_dc = 6 * 10 ** (-7)   # given in the paper, discussed with range from 10^-8 to 10^-5
e_2 = -torch.log(Y_0)

# e_3
# Misalignment error probability
# 4*1e-2          # given in the paper, discussed with range from 0 to 0.1
e_mis = torch.tensor(5e-3, device=device)  # Misalignment error probability # Given in the paper, discussed with range from 0 to 0.1 
e_d = e_mis
# 0.026 
e_3 = e_d * 100

# e_4
# Detected events
n_X_values = torch.tensor([10 ** s for s in range(6, 11)], dtype=torch.float32, device=device)  # Detected events
N = n_X_values
e_4 = torch.log10(N)

# Prepare input combinations
# inputs = [(L, n_X) for L in torch.linspace(0.1, 200, 100) for n_X in n_X_values]

## Other Parameters

In [7]:
# Ensure the device is properly set
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Initialize parameters
alpha = torch.tensor(0.2, device=device)  # Attenuation coefficient (dB/km), given in the paper
eta_Bob = torch.tensor(0.1, device=device)  # Detector efficiency, given in the paper
P_ap = torch.tensor(0.0, device=device)  # After-pulse probability
f_EC = torch.tensor(1.16, device=device)  # Error correction efficiency given in the paper, range around 1.1
epsilon_sec = torch.tensor(1e-10, device=device)  # Security error, scalar value
epsilon_cor = torch.tensor(1e-15, device=device)  # Correlation error
n_event = torch.tensor(1, device=device)  # For single-photon events
kappa = torch.tensor(1e-15, device=device)  # Security parameter given in the paper

## Optimal Parameters

In [8]:
# Define optimal parameters as PyTorch tensors and move them to the correct device
mu_1 = torch.tensor(6e-1, device=device)  # p_1
mu_2 = torch.tensor(2e-1, device=device)  # p_2
mu_3 = torch.tensor(2e-4, device=device)  # Fixed value for the third intensity
mu_k_values = torch.tensor([mu_1, mu_2, mu_3], device=device)  # Intensity levels

P_mu_1 = torch.tensor(0.65, device=device)  # p_3
P_mu_2 = torch.tensor(0.3, device=device)   # p_4
P_mu_3 = 1 - P_mu_1 - P_mu_2  # Automatically calculate third probability
P_mu_3 = P_mu_3.to(device)  # Ensure it's on the correct device
p_mu_k_values = torch.tensor([P_mu_1, P_mu_2, P_mu_3], device=device)  # Probabilities

P_X_value = torch.tensor(5e-3, device=device)  # p_5
P_Z_value = 1 - P_X_value  # Automatically calculate
P_Z_value = P_Z_value.to(device)  # Ensure it's on the correct device

# Prepare parameters tuple for use
params = (mu_1, mu_2, P_mu_1, P_mu_2, P_X_value)
# def optimal_parameters(params):
#     mu_1, mu_2, P_mu_1, P_mu_2, P_X_value = params
#     mu_3 = 2e-4
#     P_mu_3 = 1 - P_mu_1 - P_mu_2
#     P_Z_value = 1 - P_X_value
#     mu_k_values = to_device(torch.tensor([mu_1, mu_2, mu_3], device=device)
#     return params, mu_3, P_mu_3, P_Z_value, mu_k_values

In [9]:
from QKD_Functions_Torch import (
    to_device,
    # to_tensor,
    calculate_factorial,
    calculate_tau_n,
    calculate_eta_ch,
    calculate_eta_sys,
    calculate_D_mu_k,
    calculate_n_X_total,
    calculate_N,
    calculate_n_Z_total,
    calculate_e_mu_k,
    calculate_e_obs,
    calculate_h,
    calculate_lambda_EC,
    calculate_sqrt_term,
    calculate_tau_n,
    calculate_n_pm, 
    calculate_S_0,
    calculate_S_1,
    calculate_m_mu_k,
    calculate_m_pm,
    calculate_v_1,
    calculate_gamma,
    calculate_Phi,
    calculate_LastTwoTerm,
    calculate_l,
    calculate_R,
    experimental_parameters,
    other_parameters,
    calculate_key_rates_and_metrics,
    penalty, 
    objective,
)

In [10]:
# Check for MPS (Metal Performance Shaders) support
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS for GPU acceleration.")
else:
    device = torch.device("cpu")
    print("MPS is not available. Using CPU.")

# Define metric information
metric_info = {
    "eta_ch": ("Channel Transmittance $\eta_{ch}$", "Channel Transmittance vs Fiber Length"),
    "S_X_0": ("$S_{X_0}$", "Single-photon Events $S_{X_0}$ vs Fiber Length"),
    "S_Z_0": ("$S_{Z_0}$", "Single-photon Events $S_{Z_0}$ vs Fiber Length"),
    "tau_0": ("Tau 0", "Tau 0 vs Fiber Length"),
    "e_obs_X": ("$e_{obs,X}$", "Observed Error Rate $e_{obs,X}$ vs Fiber Length"),
    "S_X_1": ("$S_{X_1}$", "Single-photon Events $S_{X_1}$ vs Fiber Length"),
    "S_Z_1": ("$S_{Z_1}$", "Single-photon Events $S_{Z_1}$ vs Fiber Length"),
    "tau_1": ("Tau 1", "Tau 1 vs Fiber Length"),
    "v_Z_1": ("$v_{Z_1}$", "$v_{Z_1}$ vs Fiber Length"),
    "gamma": ("Gamma", "Gamma vs Fiber Length"),
    "Phi_X": ("$\Phi_{X}$", "$\Phi_{X}$ vs Fiber Length"),
    "binary_entropy_Phi": ("Binary Entropy of $\Phi$", "Binary Entropy of $\Phi$ vs Fiber Length"),
    "lambda_EC": ("Lambda EC", "Lambda EC vs Fiber Length"),
    "l_calculated": ("Calculated Secret Key Length $l$", "Calculated Secret Key Length $l$ vs Fiber Length"),
    "key_rates": ("Secret Key Rate per Pulse (bit)", "Secret Key Rate vs Fiber Length"),
}

# Ensure all parameters and constants are moved to the same device
alpha = torch.tensor(0.2, device=device)           # Attenuation coefficient (dB/km)
eta_Bob = torch.tensor(0.1, device=device)         # Detector efficiency
P_dc_value = torch.tensor(6e-7, device=device)     # Dark count probability
epsilon_sec = torch.tensor(1e-10, device=device)   # Security error
epsilon_cor = torch.tensor(1e-15, device=device)   # Correlation error
f_EC = torch.tensor(1.16, device=device)           # Error correction efficiency
e_mis = torch.tensor(0.005, device=device)         # Misalignment error
P_ap = torch.tensor(0.0, device=device)            # After-pulse probability
n_event = torch.tensor(1, device=device)           # Single-photon events

# Define fiber lengths and detected events
Ls = torch.linspace(0.1, 220, 1000, device=device, dtype=torch.float32)  # Fiber lengths
n_X_values = torch.tensor([10**s for s in range(6, 11)], dtype=torch.float32, device=device)  # Detected events

# Prepare parameters (ensure they are on the correct device)
params = (mu_1.to(device), mu_2.to(device), P_mu_1.to(device), P_mu_2.to(device), P_X_value.to(device))

# Initialize results storage
results = {n_X.item(): {metric: [] for metric in metric_info.keys()} for n_X in n_X_values}
for n_X in n_X_values:
    results[n_X.item()]["fiber_lengths"] = []  # Include fiber lengths explicitly

# Main calculation loop
for n_X in n_X_values:
    for L_value in Ls:
        # Move inputs to device and ensure 1D shape for scalar tensors
        L_value = L_value.unsqueeze(0)
        n_X = n_X.unsqueeze(0)

        # Call the objective function with all inputs on the correct device
        result = objective(
            params,
            L_value,
            n_X,
            alpha,
            eta_Bob,
            P_dc_value,
            epsilon_sec,
            epsilon_cor,
            f_EC,
            e_mis,
            P_ap,
            n_event,
        )

        # Unpack results
        (
            penalized_key_rates,
            eta_ch_values,
            S_X_0_values,
            S_Z_0_values,
            S_X_1_values,
            S_Z_1_values,
            tau_0_values,
            tau_1_values,
            e_mu_k_values,
            e_obs_X_values,
            v_Z_1_values,
            gamma_results,
            Phi_X_values,
            binary_entropy_Phi_values,
            lambda_EC_values,
            l_calculated_values,
        ) = result

        # Store results
        n_X_key = n_X.item()
        results[n_X_key]["fiber_lengths"].append(L_value.item())
        results[n_X_key]["key_rates"].append(penalized_key_rates.item())
        results[n_X_key]["eta_ch"].append(eta_ch_values.item())
        results[n_X_key]["S_X_0"].append(S_X_0_values.item())
        results[n_X_key]["S_Z_0"].append(S_Z_0_values.item())
        results[n_X_key]["S_X_1"].append(S_X_1_values.item())
        results[n_X_key]["S_Z_1"].append(S_Z_1_values.item())
        results[n_X_key]["tau_0"].append(tau_0_values.item())
        results[n_X_key]["tau_1"].append(tau_1_values.item())
        results[n_X_key]["e_obs_X"].append(e_obs_X_values.item())
        results[n_X_key]["v_Z_1"].append(v_Z_1_values.item())
        results[n_X_key]["gamma"].append(gamma_results.item())
        results[n_X_key]["Phi_X"].append(Phi_X_values.item())
        results[n_X_key]["binary_entropy_Phi"].append(binary_entropy_Phi_values.item())
        results[n_X_key]["lambda_EC"].append(lambda_EC_values.item())
        results[n_X_key]["l_calculated"].append(l_calculated_values.item())

Using MPS for GPU acceleration.


NameError: name 'metric_info' is not defined

In [ ]:
for n_X, data in results.items():
    print(f"n_X: {n_X}")
    for key, value in data.items():
        print(f"  {key}: {type(value)}, Length: {len(value)}")

In [ ]:
import torch
import os
import matplotlib.pyplot as plt
import numpy as np

# Ensure `output_dir` is defined
output_dir = 'analytical_result'
os.makedirs(output_dir, exist_ok=True)

# Plotting loop
for metric, (ylabel, title) in metric_info.items():
    plt.figure(figsize=(12, 8))
    for n_X, data in results.items():
        if metric in data and "fiber_lengths" in data:  # Check both keys exist
            fiber_lengths = to_device(torch.tensor(data["fiber_lengths"], dtype=torch.float32, device=device, device=device)
            metric_values = to_device(torch.tensor(data[metric], dtype=torch.float32, device=device, device=device)
            plt.plot(fiber_lengths.cpu().numpy(), metric_values.cpu().numpy(), label=f'$n_X = {n_X:.0e}$')
    plt.xlabel('Fiber Length (km)')
    plt.ylabel(ylabel)
    plt.yscale('log')
    plt.title(title)
    plt.legend()
    plt.grid(True, which="both", linestyle='--', linewidth=0.5)

    # Save the plot
    filename = os.path.join(output_dir, f'{metric}.png')
    plt.savefig(filename)
    plt.show()

# Define the number of rows and columns for the grid of subplots
num_metrics = len(metric_info)
rows = int(np.ceil(np.sqrt(num_metrics)))  # Rows of subplots
cols = int(np.ceil(num_metrics / rows))   # Columns of subplots

# Ensure the directory exists
output_dir = 'analytical_result_torch'
os.makedirs(output_dir, exist_ok=True)

# Close all previous figures to avoid interference
plt.close('all')

fig, axes = plt.subplots(rows, cols, figsize=(24, 18), sharex=True)

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Plot each metric in its respective subplot
for idx, (metric, (ylabel, title)) in enumerate(metric_info.items()):
    ax = axes[idx]
    for n_X, data in results.items():
        if metric in data:
            fiber_lengths = to_device(torch.tensor(data["fiber_lengths"], dtype=torch.float32, device=device, device=device)
            metric_values = to_device(torch.tensor(data[metric], dtype=torch.float32, device=device, device=device)
            ax.plot(fiber_lengths.cpu().numpy(), metric_values.cpu().numpy(), label=f'$n_X = {n_X:.0e}$')
    ax.set_xlabel('Fiber Length (km)')
    ax.set_ylabel(ylabel)
    ax.set_yscale('log')  # Log scale for y-axis
    ax.set_title(title)
    ax.legend()
    ax.grid(True, which="both", linestyle='--', linewidth=0.5)

# Hide any unused subplots
for idx in range(len(metric_info), len(axes)):
    fig.delaxes(axes[idx])

# Adjust layout to prevent overlap
plt.tight_layout()

# Save the grid of plots before showing it
filename = os.path.join(output_dir, 'all_metrics_grid.png')
plt.savefig(filename)

# Display the plot
plt.show()


In [10]:
# # Parameter values
# print(f"Fiber length range (L): {min(Ls)} km to {max(Ls)} km")
# print(f"Attenuation coefficient (alpha): {alpha}")
# print(f"Dark count probability (P_dc): {P_dc_value}")
# print(f"Misalignment error probability (e_mis): {e_mis}")
# print(f"After pulse probability (P_ap): {P_ap}")
# print(f"System transmittance (eta_sys_values): {eta_sys_values}")
# print(f"Channel transmittance (eta_ch_values): {eta_ch_values}")
# print(f"Detector efficiency (eta_Bob): {eta_Bob}")
# print(f"Secrecy parameter (epsilon_sec): {epsilon_sec}")
# print(f"Correctness parameter (epsilon_cor): {epsilon_cor}")
# print(f"Error correction efficiency (f_EC): {f_EC}")
# print(f"Detected events in X basis (n_X): {n_X}")
# print(f"Detected events in Z basis (n_Z_mu_values): {n_Z_mu_values}")
# print(f"Probability of X basis (P_X_value): {P_X_value}")
# print(f"Probability of Z basis (P_Z_value): {P_Z_value}")
# print(f"Sum of intensities probabilities (P_mu_1 + P_mu_2 + P_mu_3): {P_mu_1 + P_mu_2 + P_mu_3}")
# print(f"Total probability of detection in X basis (sum_P_det_mu_X): {sum_P_det_mu_X}")
# print(f"Total probability of detection in Z basis (sum_P_det_mu_Z): {sum_P_det_mu_Z}")
# print(f"Total events in X basis (n_X_values): {n_X_mu_k_values}")
# print(f"Total events in Z basis (n_X_total): {n_X_total}\n")

# print(f"Total errors in X basis (m_X_values): {m_X_mu_values}")
# print(f"Total errors in X basis (m_X_total): {m_X_total}")
# print(f"Total errors in Z basis (m_Z_values): {m_Z_mu_values}")
# print(f"Total errors in Z basis (m_Z_total): {m_Z_total}\n")

# # Tau and Square Root Terms
# print(f"Poisson probability for zero photons (tau_0_values): {tau_0_values}")
# print(f"Poisson probability for one photon (tau_1_values): {tau_1_values}")
# print(f"Square root term of n in X basis (sqrt_term_n_X): {sqrt_term_n_X}")
# print(f"Square root term of n in Z basis (sqrt_term_n_Z): {sqrt_term_n_Z}\n")
# print(f"Square root term of m in X basis (sqrt_term_m_X): {sqrt_term_m_X} \n")
# print(f"Square root term of m in Z basis (sqrt_term_m_Z): {sqrt_term_m_Z} \n")

# # Vacuum and Single-Photon Events for X and Z basis
# print(f"Vacuum events in X basis (S_X_0_values_list): {min(S_X_0_values_list)} to {max(S_X_0_values_list)}")
# print(f"Single-photon events in X basis (S_X_1_values_list): {min(S_X_1_values_list)} to {max(S_X_1_values_list)}")
# print(f"Vacuum events in Z basis (S_Z_0_values_list): {min(S_Z_0_values_list)} to {max(S_Z_0_values_list)}")
# print(f"Single-photon events in Z basis (S_Z_1_values_list): {min(S_Z_1_values_list)} to {max(S_Z_1_values_list)}\n")

# # Intensity-Level-Specific Parameters
# for i, (mu, P_mu, D_mu, e_mu, n_X_mu, n_Z_mu) in enumerate(
#         zip(mu_k_values, p_mu_k_values, D_mu_k_values, e_mu_k_values, 
#             [n_X_mu_1, n_X_mu_2, n_X_mu_3], [n_Z_mu_1, n_Z_mu_2, n_Z_mu_3]), 1):
#     print(f"\nIntensity Level {i}: μ_{i} = {mu}")
#     print(f"Probability for μ_{i} (P_μ_{i}): {P_mu}")
#     print(f"Detection probability (D_μ_{i}): {D_mu}")
#     print(f"Error rate (e_μ_{i}): {e_mu}")
#     print(f"Events in X basis (n_X_μ_{i}): {n_X_mu}")
#     print(f"Events in Z basis (n_Z_μ_{i}): {n_Z_mu}\n")
    
# # Error bounds in X and Z basis by intensity
# for i, (m_X_mu, m_Z_mu, m_plus_X, m_minus_X, m_plus_Z, m_minus_Z) in enumerate(
#         zip([m_X_mu_1, m_X_mu_2, m_X_mu_3], [m_Z_mu_1, m_Z_mu_2, m_Z_mu_3],
#             [m_plus_X_mu_1, m_plus_X_mu_2, m_plus_X_mu_3], [m_minus_X_mu_1, m_minus_X_mu_2, m_minus_X_mu_3],
#             [m_plus_Z_mu_1, m_plus_Z_mu_2, m_plus_Z_mu_3], [m_minus_Z_mu_1, m_minus_Z_mu_2, m_minus_Z_mu_3]), 1):
#     print(f"Errors in X basis (m_X_μ_{i}): {m_X_mu}")
#     print(f"Bounds in X basis (m_plus_X_μ_{i}, m_minus_X_μ_{i}): ({m_plus_X}, {m_minus_X})")
#     print(f"Errors in Z basis (m_Z_μ_{i}): {m_Z_mu}")
#     print(f"Bounds in Z basis (m_plus_Z_μ_{i}, m_minus_Z_μ_{i}): ({m_plus_Z}, {m_minus_Z})\n") 

# # Single-photon parameters
# print(f"Single-photon events in Z basis (v_Z_1_values): {v_Z_1_values}")
# print(f"Ratio of v_Z_1 to S_Z_1 (v_Z_1_values / S_Z_1_values): {v_Z_1_values / S_Z_1_values}")

# # Security and Key Rate Calculations
# print(f"Observed error rate in X basis (e_obs_X_values): {e_obs_X_values}")
# print(f"Error correction efficiency (lambda_EC_values): {lambda_EC_values}\n")

# print(f"Gamma (security adjustment) (gamma_results): {gamma_results}")
# print(f"Phi_X (binary entropy parameter) (Phi_X_values): {Phi_X_values}")
# print(f"Binary entropy value (binary_entropy_Phi_values): {binary_entropy_Phi_values}")
# print(f"Secret key length (l_calculated_values): {l_calculated_values}")
# print(f"Secret key rate per pulse (R) (key_rates_list): {min(key_rates_list)} to {max(key_rates_list)}")

# print(f"Vacuum events in X basis (S_X_0_values_list): {min(S_X_0_values_list)} to {max(S_X_0_values_list)}")
# print(f"Single-photon events in X basis (S_X_1_values_list): {min(S_X_1_values_list)} to {max(S_X_1_values_list)}")